<a href="https://colab.research.google.com/github/imPhacko/PublicProjects/blob/main/AdvancedRNNTechniques_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelling long-term dependencies

https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filename = "/content/drive/MyDrive/2022/Lecture10/metai.txt"
raw_text = open(filename, "r", encoding="utf-8").read()
raw_text = raw_text.lower()
print(raw_text[:200])

jau saulelė vėl atkopdama budino svietą 
ir žiemos šaltos trūsus pargriaudama juokės. 
šalčių pramonės su ledais sugaišti pagavo, 
ir putodams sniegs visur į nieką pavirto. 
tuo laukus orai drungni ga


In [ ]:
characters = sorted(list(set(raw_text)))

char_to_int = {character: integer for integer, character in enumerate(characters)}

for character in characters:
    print(f"{repr(character)}: {char_to_int[character]}")

print(f"Text length: {len(raw_text)}")
print(f"Dictionary length: {len(char_to_int)}")

'\n': 0
' ': 1
'!': 2
'"': 3
"'": 4
'(': 5
')': 6
',': 7
'-': 8
'.': 9
':': 10
';': 11
'?': 12
'a': 13
'b': 14
'c': 15
'd': 16
'e': 17
'g': 18
'i': 19
'j': 20
'k': 21
'l': 22
'm': 23
'n': 24
'o': 25
'p': 26
'r': 27
's': 28
't': 29
'u': 30
'v': 31
'y': 32
'z': 33
'ą': 34
'č': 35
'ė': 36
'ę': 37
'į': 38
'š': 39
'ū': 40
'ų': 41
'ž': 42
Text length: 134183
Dictionary length: 43


In [ ]:
import numpy as np

data = np.zeros((len(raw_text), len(char_to_int)))
int_data = np.zeros((len(raw_text),), dtype=np.int32)

for idx, character in enumerate(raw_text):
    data[idx, char_to_int[character]] = 1.0
    int_data[idx] = char_to_int[character]

In [ ]:
int_to_char = {integer: character for integer, character in enumerate(characters)}
def onehot_to_character(onehot):
    category = np.argmax(onehot)
    return int_to_char[category]

input_data = int_data[:-1]
output_data = data[1:]

sequence_length = 128

X = []
y = []
for i in range(0, len(input_data) - sequence_length, sequence_length):
    X.append(input_data[i:i + sequence_length])
    y.append(output_data[i:i + sequence_length])

X = np.asarray(X)
y = np.asarray(y)
print(X.shape, y.shape)

(1048, 128) (1048, 128, 43)


In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Input((None,)),
    tf.keras.layers.Embedding(43, 128),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(43, activation="softmax")
])

model.summary()

model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(lr=0.01), metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         5504      
                                                                 
 gru (GRU)                   (None, None, 128)         99072     
                                                                 
 gru_1 (GRU)                 (None, None, 128)         99072     
                                                                 
 dense (Dense)               (None, None, 43)          5547      
                                                                 
Total params: 209,195
Trainable params: 209,195
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
def step_decay(epoch):
    lr = 0.01
    if epoch > 20:
        lr = 0.001
    if epoch > 40:
        lr = 0.0005
    print(f"Current learning rate: {lr} at {epoch}")
    return lr

model.fit(X, y, epochs=60, batch_size=64, verbose=1, callbacks=[tf.keras.callbacks.LearningRateScheduler(step_decay)])

Current learning rate: 0.01 at 0
Epoch 1/60
17/17 [==============================] - 9s 73ms/step - loss: 3.1292 - accuracy: 0.1663 - lr: 0.0100
Current learning rate: 0.01 at 1
Epoch 2/60
17/17 [==============================] - 1s 73ms/step - loss: 2.4767 - accuracy: 0.2849 - lr: 0.0100
Current learning rate: 0.01 at 2
Epoch 3/60
17/17 [==============================] - 1s 72ms/step - loss: 2.1955 - accuracy: 0.3599 - lr: 0.0100
Current learning rate: 0.01 at 3
Epoch 4/60
17/17 [==============================] - 1s 72ms/step - loss: 1.9950 - accuracy: 0.4058 - lr: 0.0100
Current learning rate: 0.01 at 4
Epoch 5/60
17/17 [==============================] - 1s 71ms/step - loss: 1.8455 - accuracy: 0.4416 - lr: 0.0100
Current learning rate: 0.01 at 5
Epoch 6/60
17/17 [==============================] - 1s 71ms/step - loss: 1.7270 - accuracy: 0.4737 - lr: 0.0100
Current learning rate: 0.01 at 6
Epoch 7/60
17/17 [==============================] - 1s 71ms/step - loss: 1.6300 - accuracy: 0.500

In [ ]:
for character in characters:
    occurances = raw_text.count(character)
    freq = occurances / len(raw_text)

    print(f"{repr(character)}: {freq:.4f}")

'\n': 0.0221
' ': 0.1473
'!': 0.0016
'"': 0.0004
"'": 0.0003
'(': 0.0001
')': 0.0001
',': 0.0206
'-': 0.0008
'.': 0.0070
':': 0.0002
';': 0.0018
'?': 0.0010
'a': 0.0963
'b': 0.0135
'c': 0.0003
'd': 0.0322
'e': 0.0357
'g': 0.0169
'i': 0.1007
'j': 0.0126
'k': 0.0441
'l': 0.0233
'm': 0.0255
'n': 0.0347
'o': 0.0338
'p': 0.0281
'r': 0.0434
's': 0.0709
't': 0.0393
'u': 0.0429
'v': 0.0213
'y': 0.0097
'z': 0.0011
'ą': 0.0066
'č': 0.0035
'ė': 0.0168
'ę': 0.0037
'į': 0.0041
'š': 0.0136
'ū': 0.0064
'ų': 0.0073
'ž': 0.0084


In [ ]:
def text_to_onehot(text):
    onehot = np.zeros((len(text), len(char_to_int)))
    for idx, character in enumerate(text):
        onehot[idx, char_to_int[character]] = 1.0

    return onehot

def text_to_integer(text):
    int_text = np.zeros((len(text),), dtype=np.int32)
    for idx, character in enumerate(text):
        int_text[idx] = char_to_int[character]

    return int_text

In [ ]:
sample_text = "studentai "

for _ in range(256):
    int_text = text_to_integer(sample_text)
    int_text = int_text[np.newaxis]

    predictions = model.predict(int_text)
    next_character_probabilities = predictions[0, -1]
    next_character = onehot_to_character(next_character_probabilities)

    sample_text += next_character

print(sample_text)

studentai 
gėdėtis vis ir kaimynai su padėjimu dievo 
iš bėdos pasidaręs ant laukų pasirodė 
ir ponai kožnam vis pasirodė. 
,,kas tikt dievs vis ir kaimynai su pakulienė 
vis šlapjurgis su laukus bei patalą kopa. 
o paskui jisai mus ir kiaulių daug parduot ant audus


In [ ]:
rng = np.random.RandomState(0)

sample_text = "studentai "

for _ in range(256):
    int_text = text_to_integer(sample_text)
    int_text = int_text[np.newaxis]

    predictions = model.predict(int_text)
    next_character_probabilities = predictions[0, -1]
    most_probable_indices = np.argsort(next_character_probabilities)[-5:][::-1]

    most_probable_probabilities = next_character_probabilities[most_probable_indices]
    most_probable_probabilities = most_probable_probabilities / np.sum(most_probable_probabilities)
    next_character_position = rng.choice(most_probable_indices, p=most_probable_probabilities)
    next_character = int_to_char[next_character_position]


    #possible_values = list(range(43))
    #next_character_position = rng.choice(possible_values, p=next_character_probabilities)
    #next_character = int_to_char[next_character_position]

    sample_text += next_character

print(sample_text)

studentai 
kad mes prasotais ubėda stopsodams 
ir su šaukdams dar mūsų būrus pastum savo namams 
irgi braugiaus savo vandi, kasdien ausis 
ir su ponu baisu mūs, gaidžiai prisiėdę 
ar kad sviets tarp jų tylom mūsų pakavot 
ir nesibastės pavasarį baikit, 
alt gaišinks
